In [ ]:
topic_name = "stress_test_json"

In [ ]:
import json


class Config:
    def __init__(self):
        config = json.loads(open("../config.json").read())
        self.kc_endpoint = config["kc_endpoint"]
        self.kc_api_key = config["kc_api_key"]
        self.kc_api_secret = config["kc_api_secret"]
        self.sr_endpoint = config["sr_endpoint"]
        self.sr_api_key = config["sr_api_key"]
        self.sr_api_secret = config["sr_api_secret"]


c = Config()

In [ ]:
import orjson
from IPython.display import clear_output

from confluent_kafka import Producer

import datagen


conf = {
    "bootstrap.servers": c.kc_endpoint,
    "security.protocol": "SASL_SSL",
    "sasl.mechanisms": "PLAIN",
    "sasl.username": c.kc_api_key,
    "sasl.password": c.kc_api_secret,
}
producer = Producer(conf)


def produce_messages(num_messages: int = 5):
    messages_sent = []

    for i in range(num_messages):
        key = datagen.generate_key()
        headers = datagen.generate_headers()
        value = datagen.generate_large_datatypes(144 * 1024)
        serialized_value = orjson.dumps(value)
        producer.produce(
            topic_name,
            key=key,
            value=serialized_value,
            headers=headers,
            on_delivery=lambda err, msg: print(f"{err=}") if err else print(f"{msg=}"),
        )

        if i < 5:
            # only keep a sample of generated messages
            messages_sent.append(
                {
                    "key": key,
                    "value": value,
                    "headers": headers,
                    "topic": topic_name,
                    "serialized_value": None,
                }
            )

        if i % 100 == 0:
            producer.flush()
        clear_output()

    return messages_sent


messages = produce_messages(5_000_000)

In [ ]:
len(json.dumps(messages[0])), messages[0]